In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
sns.set_theme(style='whitegrid', context='talk')

In [ ]:
# bloco: leitura e plot de tempo (substitui o bloco anterior que usava header=None e nomes diferentes)
df = pd.read_csv('results.csv', header=0)
# mapear nomes do CSV para os nomes esperados pelo notebook
df = df.rename(columns={'expansions': 'expanded', 'elapsed': 'time', 'steps': 'solution_len'})
df['time'] = pd.to_numeric(df['time'], errors='coerce')

pivot = df.pivot_table(index='instance', columns='algorithm', values='time', aggfunc='mean')

def instance_key(s):
    parts = s.split('-')
    try:
        return tuple(int(p) for p in parts)
    except:
        return tuple(parts)

ordered_instances = sorted(pivot.index, key=instance_key)
pivot = pivot.reindex(ordered_instances)

algorithms = list(pivot.columns)
n_algs = len(algorithms)
n_inst = len(pivot.index)
x = np.arange(n_inst)
bar_width = 0.8 / max(n_algs, 1)

min_pos = df[df['time'] > 0]['time'].min()
eps = (min_pos if pd.notna(min_pos) else 1e-6) * 1e-3

colors = sns.color_palette('tab10', n_colors=n_algs)

plt.figure(figsize=(14, 6))
for i, alg in enumerate(algorithms):
    raw = pivot[alg].values
    mask = np.isnan(raw) | (raw == 0)
    vals = np.where(mask, eps, raw)
    bars = plt.bar(x + i*bar_width, vals, width=bar_width, label=alg,
                   color=colors[i], edgecolor='k', linewidth=0.6)
    for j, b in enumerate(bars):
        if mask[j]:
            b.set_alpha(0.35)
            b.set_hatch('//')
        else:
            v = raw[j]
            txt = f"{v:.3f}s" if v < 1 else f"{v:.2f}s"
            plt.text(b.get_x() + b.get_width()/2, b.get_height()*1.05, txt,
                     ha='center', va='bottom', fontsize=8, rotation=90)

plt.yscale('log')
plt.xlabel('Instance')
plt.ylabel('Execution time (s) — log scale')
plt.title('Plot 1: Execution Time per Instance for Each Algorithm')
plt.xticks(x + (n_algs-1)*bar_width/2, pivot.index, rotation=45, ha='right')
plt.legend(title='Algorithm', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.grid(which='both', axis='y', alpha=0.4)
plt.tight_layout()
plt.savefig('./graphs/plot_time_per_instance_per_algorithm.png',
            dpi=300, bbox_inches='tight')
plt.close()
print('Saved: plot1_time_per_instance_per_algorithm.png')

Saved: plot1_time_per_instance_per_algorithm.png


In [5]:
# bloco: nodes expanded (reusa df já carregado)
df['expanded'] = pd.to_numeric(df['expanded'], errors='coerce')

pivot2 = df.pivot_table(index='instance', columns='algorithm', values='expanded', aggfunc='mean')

ordered_instances = sorted(pivot2.index, key=instance_key)
pivot2 = pivot2.reindex(ordered_instances)

algorithms = list(pivot2.columns)
n_algs = len(algorithms)
n_inst = len(pivot2.index)
x = np.arange(n_inst)
bar_width = 0.8 / max(n_algs, 1)

min_pos = df[df['expanded'] > 0]['expanded'].min()
eps = (min_pos if pd.notna(min_pos) else 1) * 1e-3

colors = sns.color_palette('tab10', n_colors=n_algs)

plt.figure(figsize=(14, 6))
for i, alg in enumerate(algorithms):
    raw = pivot2[alg].values
    mask = np.isnan(raw) | (raw == 0)
    vals = np.where(mask, eps, raw)
    bars = plt.bar(x + i*bar_width, vals, width=bar_width, label=alg,
                   color=colors[i], edgecolor='k', linewidth=0.6)
    for j, b in enumerate(bars):
        if mask[j]:
            b.set_alpha(0.35)
            b.set_hatch('//')
        else:
            v = raw[j]
            plt.text(b.get_x() + b.get_width()/2, b.get_height()*1.05, f"{v:,.0f}",
                     ha='center', va='bottom', fontsize=8, rotation=90)

plt.yscale('log')
plt.xlabel('Instance')
plt.ylabel('Nodes expanded (memory) — log scale')
plt.title('Plot 2: Nodes Expanded per Instance for Each Algorithm')
plt.xticks(x + (n_algs-1)*bar_width/2, pivot2.index, rotation=45, ha='right')
plt.legend(title='Algorithm', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.grid(which='both', axis='y', alpha=0.4)
plt.tight_layout()
plt.savefig('./graphs/plot_nodes_expanded_memory.png', dpi=300, bbox_inches='tight')
plt.close()
print('Saved: plot_nodes_expanded_memory.png')

Saved: plot_nodes_expanded_memory.png
